In [81]:
import random

# Variables

In [118]:
nucleotides = ['A', 'C', 'T', 'G']
length = 10
m = 4
g = 4
k = 1

# Number of splits from each string at each generation (i.e., one bacteria splitting in half to become two)
s = 2

### Generate our starting population of m ancestor strings

In [119]:
ancestors = []
for i in range(m):
    rand_ancestor = ''
    for j in range(length):
        rand_nucleotide = random.choice(nucleotides)
        rand_ancestor = rand_ancestor + rand_nucleotide
    ancestors.append((rand_ancestor, i))
ancestors

[('GTTCAAAACA', 0), ('CCCTCGAGGG', 1), ('TAAGAGAGTC', 2), ('GGCATCCCCT', 3)]

In [120]:
def mutate_string(string, k):
    """ Mutate k locations in the given string, returning this new mutated string"""
    indices_to_replace = random.sample(range(0, len(string)), k)
    new_string = ''
    for i in range(len(string)):
        if i in indices_to_replace:
            new_string = new_string + random.choice(nucleotides)
        else:
            new_string = new_string + string[i]
            
    return new_string
  
# Test function...
mutate_string('ACTG', 1)

'ACTC'

### Generate descendants!

In [121]:
# For each string, for the number of generations specified, split each string into "s" strings all with "k" mutations 
# compared to the parent string. 
population = ancestors
for generations in range(g):
    new_population = []
    for string, origin in population:
        for splits in range(s):
            new_string = mutate_string(string, k) 
            new_population.append((new_string, origin))
    population = new_population
    
# (population contains tuples that also indicate which of the original 'm' ancestors each descendant came from)

In [122]:
for descendant in population:
    print(descendant[0])

ATTAAAAACG
GTTAAAATCG
GTTAAACACG
GTTAATGACG
GTTAAAGTCA
GTTAAAATCG
GTTAATAACA
GTTAAAAACA
CTTCAAGACA
CTTCAAAGCA
CTCCAAAGTA
CACCAAAATA
CTTCTAAACA
CTTCAAAGCA
ATTCAAAACT
ACTCAAAACT
CTCATGAAGG
CTTACGAAGG
TTCACGACGG
ATCACGACGG
CGCTCCAGGG
CGCTCCAGGG
CTCTCCAGGG
TTCTCCAGGG
CATTCGCAGG
CACTGGCAGG
CACTCGAACG
CACTCAAACG
TTCTCGAGGC
CTCTCGAGGC
CACGCGATGC
CACGCCAGGC
TAAGACTGAC
TAAGCGTGAC
TAAGAGTGAC
TAACAGTGAC
TAAGAATGAC
TATGAATGAC
AGAGAATGTC
TGAAAATGTC
AGAGAGACTC
TAAGAGACTC
TAAGCTACTC
TAAGATACCC
TAAGCAAGTC
TAAGCGAGTG
TATGAGAGTT
TCAGAGAGTT
GCAATCCCAT
GCAATCCCAT
GTACTCCCAT
GGCCTCCCAT
TGAATGCCCT
TGAATCCCCT
TGACTCCCCT
TGAATCCGCT
GATATGCCCA
GGTATGCTCA
GGGATGCCCA
GGCATGCCCA
GACCTCCCCA
GTCCTCTCCA
GCCCTCCTCA
GGCCTCCACA
